In [ ]:
!df -h #Checks Storage Spaces 

Filesystem        Size  Used Avail Use% Mounted on
overlay           7.9T  6.0T  1.9T  77% /
tmpfs              64M     0   64M   0% /dev
shm                14G     0   14G   0% /dev/shm
/dev/loop1         20G   68K   20G   1% /kaggle/lib
/dev/sda1         122G  4.5G  118G   4% /opt/bin
/dev/mapper/snap  7.9T  6.0T  1.9T  77% /etc/hosts
tmpfs              16G     0   16G   0% /proc/acpi
tmpfs              16G     0   16G   0% /proc/scsi
tmpfs              16G     0   16G   0% /sys/firmware


# 1️⃣ **Setup and Installing Dependencies**

In [4]:
!pip install ffmpeg-python transformers datasets torchaudio huggingface_hub huggingsound

INFO: pip is looking at multiple versions of huggingsound to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingsound to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 6.9 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 490.2 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 449.7 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 6.8 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━

In [5]:
!pip install deep-translator
!pip install sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 49.6 MB/s eta 0:00:00:00:0100:01


In [ ]:
# In case, there are CUDA issues, run this
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Found existing installation: torch 1.12.1
Uninstalling torch-1.12.1:
  Successfully uninstalled torch-1.12.1
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 0.12.1
Uninstalling torchaudio-0.12.1:
  Successfully uninstalled torchaudio-0.12.1
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 63.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 77.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.5 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 2.0 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 9.9 MB/s eta 0:00:000:00:0100:01
     ━━━━━━━━━

In [7]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 4.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 44.8 MB/s eta 0:00:0000:0100:01


# 2️⃣ **Import Required Modules**

In [ ]:
import os
import time
import torch
import ffmpeg
import yt_dlp
from transformers import pipeline
from deep_translator import GoogleTranslator
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from huggingface_hub import HfApi, HfFolder, Repository, notebook_login

In [9]:
pwd

'/kaggle/working'

In [10]:
mkdir downloads

In [11]:
ls

downloads/


In [12]:
cd downloads

/kaggle/working/downloads


# 3️⃣ **Downloading Video from YouTube**

In [13]:
def download_youtube_video(youtube_url, download_path):
    ydl_opts = {
        'format': 'bestvideo+bestaudio/best',
        'outtmpl': f"{download_path}/%(title)s.%(ext)s"
    }
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([youtube_url])
        print(f"✅ Video downloaded successfully!")
    except Exception as e:
        print(f"❌ Error: {e}")

In [14]:
# def download_youtube_video(youtube_url, download_path):
#     os.makedirs(download_path, exist_ok=True)
    
#     command = f'yt-dlp -f "bestvideo[ext=mp4]+bestaudio[ext=m4a]" --merge-output-format mp4 -o "{download_path}/%(title)s.%(ext)s" "{youtube_url}"'
    
#     os.system(command)
#     print("✅ Downloaded and merged as MP4!")

In [15]:
youtube_link = input("Enter YouTube video link: ")
# save_path = input("Enter folder path to save video: ")
savePath = r"/kaggle/working/downloads"

Enter YouTube video link:  https://youtu.be/NiKtZgImdlY?feature=shared


In [16]:
download_youtube_video(youtube_link, savePath)

[youtube] Extracting URL: https://youtu.be/NiKtZgImdlY?feature=shared
[youtube] NiKtZgImdlY: Downloading webpage
[youtube] NiKtZgImdlY: Downloading tv client config
[youtube] NiKtZgImdlY: Downloading player 9c6dfc4a
[youtube] NiKtZgImdlY: Downloading tv player API JSON
[youtube] NiKtZgImdlY: Downloading ios player API JSON
[youtube] NiKtZgImdlY: Downloading m3u8 information
[info] NiKtZgImdlY: Downloading 1 format(s): 398+251
[download] Destination: /kaggle/working/downloads/The danger of silence ｜ Clint Smith ｜ TED.f398.mp4
[download] 100% of   11.02MiB in 00:00:00 at 22.90MiB/s    
[download] Destination: /kaggle/working/downloads/The danger of silence ｜ Clint Smith ｜ TED.f251.webm
[download] 100% of    3.41MiB in 00:00:00 at 5.31MiB/s   
[Merger] Merging formats into "/kaggle/working/downloads/The danger of silence ｜ Clint Smith ｜ TED.webm"
Deleting original file /kaggle/working/downloads/The danger of silence ｜ Clint Smith ｜ TED.f398.mp4 (pass -k to keep)
Deleting original file /ka

## *Conversion to .mp4 Format*

"yt-dlp" downloads video in '.webm' format. So it needs to be converted to '.mp4' for further processing

In [19]:
def convert_webm_to_mp4(input_path):
    output_path = input_path.replace(".webm", ".mp4")
    os.system(f"ffmpeg -i \"{input_path}\" -c:v libx264 -preset fast -crf 23 -c:a aac -b:a 192k \"{output_path}\"")
    print(f"✅ Converted to MP4: {output_path}")
    return output_path

webm_file = r"/kaggle/working/downloads/The danger of silence ｜ Clint Smith ｜ TED.webm"
mp4_file = convert_webm_to_mp4(webm_file)

✅ Converted to MP4: /kaggle/working/downloads/The danger of silence ｜ Clint Smith ｜ TED.mp4


# 4️⃣ **Setup the Video Path**

In [20]:
# yt = r"/kaggle/input/tedx-video/woody-roseland--tedxmilehigh---a-one-minute-tedx-talk-for-the-digital-age.mp4"
# yt = r"/kaggle/working/downloads/＂Mastering Happiness： The Watermelon Lesson＂.mp4"
yt = r"/kaggle/working/downloads/The danger of silence ｜ Clint Smith ｜ TED.mp4"

In [21]:
print(os.path.exists(yt))  # Should return True


True


In [22]:
pwd

'/kaggle/working/downloads'

# 5️⃣ **Extracting Audio from Video**

In [23]:
outputAudio = r"/kaggle/working/downloads/silence.wav"

In [24]:
ffmpeg.input(yt).output(outputAudio, format="wav").run(overwrite_output=True)
print("Hey Sagar! Audio extracted successfully!")

Hey Sagar! Audio extracted successfully!


In [ ]:
# ! ffmpeg -i /kaggle/input/tedx-video/woody-roseland--tedxmilehigh---a-one-minute-tedx-talk-for-the-digital-age.mp4 -acodec pcm_s16le -ar 16000 ytAudio.wav

# 6️⃣ **English ASR with HuggingSound🎵🔊**

In [26]:
!pip install huggingsound -q 

In [27]:
from transformers import Wav2Vec2Processor
print("Transformers loaded successfully!")
from huggingsound import SpeechRecognitionModel #Upgrade CUDA, if error occurs
print("Huggingsound loaded successfully!")

Transformers loaded successfully!
Huggingsound loaded successfully!


In [ ]:
# Set the device to CUDA (for GPU-enabled environments)
device = "cuda" if torch.cuda.is_available() else "cpu"

## *Login to Hugging Face with Access Token*

In [30]:
notebook_login()

# 7️⃣ **Load the Model from 🤗/Models**

In [31]:
model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device = device)

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

# 8️⃣ **Audio Chunking (To Avoid Out of Memory Error)**

In [32]:
!pip install pydub

In [33]:
from pydub import AudioSegment
from pydub.silence import split_on_silence

In [34]:
def split_audio(audio_path, min_silence_len=1000, silence_thresh=-40, chunk_length_ms=30000):
    """
    Splits audio into smaller chunks based on silence detection or fixed intervals.

    Parameters:
        - audio_path (str): Path to input audio file.
        - min_silence_len (int): Minimum silence length (ms) to consider as a split.
        - silence_thresh (int): Silence threshold (dBFS) for detecting splits.
        - chunk_length_ms (int): Maximum chunk length in milliseconds.

    Returns:
        - List of AudioSegment chunks.
    """
    audio = AudioSegment.from_wav(audio_path)
    
    # First, try splitting based on silence
    chunks = split_on_silence(audio, min_silence_len=min_silence_len, silence_thresh=silence_thresh)

    # If silence-based splitting is not effective, split into fixed chunks
    if len(chunks) == 1:  
        chunks = [audio[i : i + chunk_length_ms] for i in range(0, len(audio), chunk_length_ms)]
    
    return chunks

In [35]:
cd ..

/kaggle/working


In [36]:
ls

downloads/


In [37]:
mkdir audioChunks

In [38]:
ls

audioChunks/  downloads/


In [39]:
# audio = r"/kaggle/working/ytAudio.wav"
audio = r"/kaggle/working/downloads/silence.wav"
# output_dir = r"/kaggle/working/audioChunks"
output_dir = r"/kaggle/working/audioChunks"

In [40]:
os.makedirs(output_dir, exist_ok=True)

chunks = split_audio(audio)

chunk_paths = []
for i, chunk in enumerate(chunks):
    chunk_filename = f"{output_dir}/chunk_{i}.wav"
    chunk.export(chunk_filename, format="wav")
    chunk_paths.append(chunk_filename)

print(f"Total Chunks: {len(chunk_paths)}")

Total Chunks: 39


In [41]:
cd audioChunks

/kaggle/working/audioChunks


In [42]:
ls

chunk_0.wav   chunk_17.wav  chunk_24.wav  chunk_31.wav  chunk_3.wav
chunk_10.wav  chunk_18.wav  chunk_25.wav  chunk_32.wav  chunk_4.wav
chunk_11.wav  chunk_19.wav  chunk_26.wav  chunk_33.wav  chunk_5.wav
chunk_12.wav  chunk_1.wav   chunk_27.wav  chunk_34.wav  chunk_6.wav
chunk_13.wav  chunk_20.wav  chunk_28.wav  chunk_35.wav  chunk_7.wav
chunk_14.wav  chunk_21.wav  chunk_29.wav  chunk_36.wav  chunk_8.wav
chunk_15.wav  chunk_22.wav  chunk_2.wav   chunk_37.wav  chunk_9.wav
chunk_16.wav  chunk_23.wav  chunk_30.wav  chunk_38.wav


In [43]:
cd ..

/kaggle/working


In [46]:
chunksFolder = r"/kaggle/working/audioChunks"

# 9️⃣ **Transcribing Each Audio Chunk**

In [48]:
# List all audio chunk files
chunk_files = sorted([os.path.join(chunksFolder, f) for f in os.listdir(chunksFolder) if f.endswith(".wav")])

In [49]:
# Transcribe each chunk
transcripts = []
for chunk in chunk_files:
    result = model.transcribe([chunk])
    transcripts.append(result[0]["transcription"])  # Extract text from result

100%|██████████| 1/1 [00:00<00:00, 21.25it/s]


## *Combine all the transcripts (of each chunk) into Single Transcript*

In [ ]:
full_transcript = " ".join(transcripts)
print("\n✅ Full Transcript:\n")
print(full_transcript)


✅ Full Transcript:

oepepeps apear doctor martin luther king junior in a nineteen-sixty-eighth beach where he reflects upon the civil rights movt state ename to understand that they don't have to be sources of shame in an effort to create a culture within my classroom where students feel safe sharing te intimacies of their own silence i have four corp principles posted on the board that sits in the front of my clas every student signs the beginning of the year read critically right-consciously speak clearly tell youre i fiund myself thinking a lot about that last tell youre and i realize that if i was going to ask my students to speak up i' was going tn have to tell mice in the end and be honest with them about times why i fail to do so i tell them that growing up as a kid in a catholic family in new orlens i was always tought that the most meaningful thing one could do was to give something upsacrificed somthing youtypically indulgent to prove to god you understand his sanctity i've 

# 1️⃣0️⃣ **Transcribe Translation**

In [51]:
# List of available languages
languages = {
    "hi": "Hindi",
    "es": "Spanish",
    "fr": "French",
    "de": "German",
    "zh": "Chinese",
    "None": "No Translation"
}

print("\nWould you like to translate the transcript? Available languages: ")
for code, lang in languages.items():
    print(f"{code} -> {lang}")


Would you like to translate the transcript? Available languages: 
hi -> Hindi
es -> Spanish
fr -> French
de -> German
zh -> Chinese
None -> No Translation


In [52]:
selected_lang = input("\nEnter language code (or 'None' to skip): ").strip().lower()


Enter language code (or 'None' to skip):  hi


In [53]:
if selected_lang in languages and selected_lang != "None":
    translated_text = GoogleTranslator(source="auto", target=selected_lang).translate(full_transcript)
    print(f"\n✅ Translated Transcript ({languages[selected_lang]}):\n")
    print(translated_text)
else:
    translated_text = full_transcript  
    print("\n✅ Skipping translation.")


✅ Translated Transcript (Hindi):

oepepeps apear डॉक्टर मार्टिन लूथर किंग जूनियर एक उन्नीस-छह-आठवें समुद्र तट में जहां वह नागरिक अधिकारों Movt राज्य को यह समझने के लिए प्रतिबिंबित करता है कि उन्हें मेरी कक्षा के भीतर एक संस्कृति बनाने के प्रयास में शर्म के स्रोत नहीं हैं, जहां छात्र छात्र हैं अपने स्वयं के चुप्पी की सुरक्षित साझा करने के लिए सुरक्षित साझा करना मेरे पास बोर्ड पर चार कॉर्प सिद्धांत हैं जो मेरे सीएलएएस के सामने बैठते हैं, यह आखिरी बार आपको बताता है और मुझे एहसास है कि अगर मैं अपने छात्रों को बोलने के लिए कहने जा रहा था तो मैं अंत में चूहों को बता रहा था और उनके साथ ईमानदार होना चाहिए कि मैं ऐसा करने में विफल क्यों हूं इसलिए मैं उन्हें बताता हूं कि बड़े हो रहे हैं। न्यू ऑरलेंस में एक कैथोलिक परिवार में एक बच्चे के रूप में, मुझे हमेशा लगता था कि सबसे सार्थक बात यह थी सब कुछ, लेकिन एक साल मैंने छोड़ दिया, सबसे मूल्यवान चीज जो मैं बलिदान कर सकता था वह मेरी खुद की आवाज थी, लेकिन यह ऐसा था जैसे मैंने हेडन को एहसास किया कि मैंने दिया था कि मैंने लंबे समय तक अपने जीवन का इतना खर

# 1️⃣1️⃣ **Transcript Summarization**

### Load the model from 🤗/Models

In [ ]:
summarizer = pipeline("summarization", model="Falconsai/text_summarization")

config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda:0


In [56]:
# Generate summary
summary = summarizer(full_transcript, max_length=500, min_length=300, do_sample=False)[0]["summary_text"]

In [57]:
print("\n✅ Summary of the Video:\n")
print(summary)


✅ Summary of the Video:

oepepeps apear doctor martin luther king junior in a nineteen-sixty-eighth beach where he reflects upon the civil rights movt state ename to understand that they don't have to be sources of shame in an effort to create a culture within my classroom . i've internalized this miss every day all around us we see the consequences of silence manifest themselves in the form of discrimination violence genocide and war... i have four corp principles posted on the board that 's - aps to tell christian that he was tn have to a 'n' eeps on the 'ename' to read critically right-consciously speak clearly tell youre - and he's the most . the . students feel . they . that . in . our . we see . as a teacher . it is . and i’ . of . their . (i .. . many . ‘sacrifice . de . my . I . have . this . but . to . how . le . on . what if . be ., i will . for a while .


# 1️⃣2️⃣ **Q&A Chatbot🤖 (Context: YouTube Transcript)**

In [58]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

## Load any Sentence-Transformer Model for Embeddings

In [ ]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# 1️⃣3️⃣ **Create FAISS Index to store mappings**

In [61]:
def create_faiss_index(transcript_chunks):
    """
    Creates a FAISS index from transcript chunks and stores mappings.
    """
    embeddings = embedding_model.encode(transcript_chunks)
    dimension = embeddings.shape[1]

    # Initialize FAISS index
    index = faiss.IndexFlatL2(dimension)  
    index.add(np.array(embeddings, dtype=np.float32))  # Convert to float32

    # Store transcript chunks in a dictionary for easy retrieval
    transcript_dict = {i: transcript_chunks[i] for i in range(len(transcript_chunks))}

    return index, transcript_dict

## Loading Model for Q&A from 🤗/Models

In [ ]:
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
qa_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [64]:
index, transcript_dict = create_faiss_index(transcripts)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [65]:
def get_relevant_text(query, index, transcript_dict, top_k=2):
    """
    Retrieves the most relevant transcript chunk(s) using FAISS.
    """
    query_embedding = embedding_model.encode([query])
    _, indices = index.search(np.array(query_embedding, dtype=np.float32), top_k)
    
    retrieved_texts = [transcript_dict[idx] for idx in indices[0]]
    return " ".join(retrieved_texts)  # Merge top-k results

In [66]:
def generate_answer(question, context):
    """
    Uses FLAN-T5 to generate an answer based on the retrieved context.
    """
    input_text = f"Context: {context} \n Question: {question}"
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

    with torch.no_grad():
        output = qa_model.generate(**inputs, max_length=100)

    return tokenizer.decode(output[0], skip_special_tokens=True)

# 1️⃣4️⃣ **Setup for the ChatBot**

In [67]:
def chat_with_video_qa(index, transcript_dict):
    """
    Interactive chatbot that answers questions using FAISS + FLAN-T5.
    """
    print("💬 Ask anything from the video! (Type 'exit' to stop)")
    
    while True:
        question = input("🧐 You: ")
        if question.lower() == "exit":
            print("👋 Exiting chatbot. Have a great day!")
            break

        # Retrieve relevant transcript chunks
        context = get_relevant_text(question, index, transcript_dict, top_k=2)

        # Generate an answer
        answer = generate_answer(question, context)
        print(f"🤖 AI: {answer}\n")

# 1️⃣5️⃣ **Start the Chat🚀**

In [68]:
chat_with_video_qa(index, transcript_dict)

💬 Ask anything from the video! (Type 'exit' to stop)


🧐 You:  What is this video about?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🤖 AI: Science/Tech



🧐 You:  What is the key takeaway from this video?


...

🧐 You:  Thanks


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

🤖 AI: i'm sorry



🧐 You:  exit


👋 Exiting chatbot. Have a great day!
